In [23]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from functools import reduce

In [24]:
%connect_info
# %qtconsole


{
  "shell_port": 49574,
  "iopub_port": 49575,
  "stdin_port": 49576,
  "control_port": 49578,
  "hb_port": 49577,
  "ip": "127.0.0.1",
  "key": "0fa750e2-25d5b67f7c8eb737b55830d0",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "python3"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-1cb3f228-cbda-4895-b28e-103b2a2ebc9b.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [25]:
pwd

'/Users/guchunhui/Documents/GNN-PPI/graphgym/pre-processing_protein_data'

In [26]:
# with open('../data/yeast/raw/reference/goldstandard1.txt') as f:
#     content = f.read().splitlines()
# content

In [27]:
gs1 = pd.read_csv('../data/yeast/raw/additional/goldstandard1.txt', header=None, index_col=False, names=['protein'])
gs2 = pd.read_csv('../data/yeast/raw/additional/goldstandard2.txt', header=None, index_col=False, names=['protein'])
gs3 = pd.read_csv('../data/yeast/raw/additional/goldstandard3.txt', header=None, index_col=False, names=['protein'])
gs4 = pd.read_csv('../data/yeast/raw/additional/goldstandard4.txt', header=None, index_col=False, names=['protein'])
all = pd.read_csv('../data/yeast/raw/additional/yeast_all.lst.txt', header=None, index_col=False, names=['protein'])

In [28]:
gs1['detected'] = 1
gs2['detected'] = 1
gs3['detected'] = 1
gs4['detected'] = 1

In [29]:
gs1

,protein,detected
0,Q0250,1
1,YAL003W,1
2,YAL005C,1
3,YAL012W,1
4,YAL015C,1
...,...,...
1998,YPR184W,1
1999,YPR187W,1
2000,YPR189W,1
2001,YPR190C,1


In [30]:
data_frames = [gs1, gs2, gs3, gs4]

In [31]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['protein'], suffixes=('', '_y'),
                                            how='outer'), data_frames).fillna(0)

In [32]:
df_merged

,protein,detected,detected_y,detected_y,detected_y
0,Q0250,1.0,0.0,0.0,0.0
1,YAL003W,1.0,1.0,1.0,1.0
2,YAL005C,1.0,1.0,1.0,1.0
3,YAL012W,1.0,1.0,1.0,0.0
4,YAL015C,1.0,1.0,0.0,0.0
...,...,...,...,...,...
2907,YPL209C,0.0,0.0,0.0,1.0
2908,YPL267W,0.0,0.0,0.0,1.0
2909,YPR008W,0.0,0.0,0.0,1.0
2910,YPR095C,0.0,0.0,0.0,1.0


In [33]:
df_merged.columns = ['protein', 'gs1', 'gs2', 'gs3', 'gs4']

In [34]:
df_merged = df_merged.set_index('protein')

In [35]:
positive_protein = df_merged[df_merged.sum(axis=1) >= 2].index.to_series()

In [39]:
df_merged[df_merged.sum(axis=1) == 0]

,gs1,gs2,gs3,gs4
protein,,,,


In [37]:
protein_with_evidence = df_merged.index.to_series()

In [16]:
all_protein = all.iloc[:, 0]

In [17]:
negative_protein = all_protein[~all_protein.isin(protein_with_evidence)]

In [82]:
yeast_orbi = pd.read_table('../data/yeast-multi-graph/raw/additional/ypd_orbi_self-yp4gte2-or-notyp4_no-tm_score_id.txt', delimiter=' ', header=None,
                         names=['ID', 'ProteinProphet_Score', 'P(K|S)', 'mRNA(M)', 'P(K|M)', 'MSpresso_Score', 'TP_or_TN_in_REFERENCEFILE'])
yeast_orbi = yeast_orbi.loc[:, ['ID', 'ProteinProphet_Score', 'mRNA(M)']]
yeast_orbi.columns = ['protein', 'protein probability', 'mRNA(M)']

In [80]:
yeast_orbi['y'] = -1
yeast_orbi.loc[yeast_orbi['protein'].isin(list(positive_protein)), 'y'] = 1
yeast_orbi.loc[yeast_orbi['protein'].isin(list(negative_protein)), 'y'] = 0

In [81]:
yeast_orbi['y'].value_counts()

y
0    1776
1    1433
Name: count, dtype: int64

For yeast-LCQ dataset

In [74]:
yeast_LCQ = pd.read_table('../data/yeast-multi-graph/raw/additional/marcottelab.org_MSpresso_roc_and_scores_MAIN_PAPER_ypd_lcq_self_yp4gte2-or-notyp4_no-tm_ypd_lcq_self_yp4gte2-or-notyp4_no-tm_score_id.txt', delimiter=' ', header=None,
                         names=['ID', 'ProteinProphet_Score', 'P(K|S)', 'mRNA(M)', 'P(K|M)', 'MSpresso_Score', 'TP_or_TN_in_REFERENCEFILE'])
yeast_LCQ = yeast_LCQ.loc[:, ['ID', 'ProteinProphet_Score', 'mRNA(M)']]
yeast_LCQ.columns = ['protein', 'protein probability', 'mRNA(M)']

In [72]:
yeast_LCQ['y'] = -1
yeast_LCQ.loc[yeast_LCQ['protein'].isin(list(positive_protein)), 'y'] = 1
yeast_LCQ.loc[yeast_LCQ['protein'].isin(list(negative_protein)), 'y'] = 0

In [73]:
yeast_LCQ['y'].value_counts()

y
0    1776
1    1433
Name: count, dtype: int64

In [58]:
protein_MS.iloc[:, 1:].to_csv('../data/yeast/raw/protein/yeast.csv', index=False)

In [59]:
protein_MS.iloc[:, 1:].columns

Index(['protein', 'protein probability', 'protein description',
       'protein length', 'percent coverage', 'tot indep spectra',
       'percent share of spectrum ids', 'peptides', 'y'],
      dtype='object')

In [162]:
positive_protein.to_csv('../data/yeast/raw/reference/positive.txt', index=False)
negative_protein.to_csv('../data/yeast/raw/reference/negative.txt', index=False)

In [163]:
positive_protein

protein
YAL003W    YAL003W
YAL005C    YAL005C
YAL012W    YAL012W
YAL015C    YAL015C
YAL016W    YAL016W
            ...   
YNL309W    YNL309W
YOR357C    YOR357C
YPL180W    YPL180W
YPR141C    YPR141C
YPR171W    YPR171W
Name: protein, Length: 1648, dtype: object

In [164]:
# interaction = pd.read_csv('../data/yeast/raw/interaction/4932.protein.links.full.v11.5.tsv', sep=' ')

In [165]:
# interaction.to_csv('../data/yeast/raw/interaction/yeast_interaction.csv', index_label=False)

In [166]:
negative_protein

0         Q0010
1         Q0017
2         Q0032
3         Q0045
4         Q0050
         ...   
6325    YPR200C
6326    YPR201W
6327    YPR202W
6328    YPR203W
6329    YPR204W
Name: protein, Length: 3424, dtype: object

In [12]:
proteins = pd.read_table('../data/yeast-multi-graph/raw/additional/marcottelab.org_MSpresso_roc_and_scores_MAIN_PAPER_ypd_lcq_self_yp4gte2-or-notyp4_no-tm_ypd_lcq_self_yp4gte2-or-notyp4_no-tm_score_id.txt', delimiter=' ', header=0,
                         names=['ID', 'ProteinProphet_Score', 'P(K|S)', 'mRNA(M)', 'P(K|M)', 'MSpresso_Score', 'TP_or_TN_in_REFERENCEFILE'])
proteins = proteins.loc[:, ['ID', 'ProteinProphet_Score', 'mRNA(M)', 'TP_or_TN_in_REFERENCEFILE']]
# proteins['y'] = np.where(proteins.TP_or_TN_in_REFERENCEFILE == 'TP', 1, 0)
proteins = proteins[['ID', 'ProteinProphet_Score', 'mRNA(M)']]
proteins.head()

,ID,protein probability,mRNA(M)
0,YDR444W,0.96,0.15
1,YHR213W,0.00,0.16
2,YLR465C,0.00,0.16
3,YDR543C,0.00,0.18
4,YCR038C,0.00,0.19


In [13]:
protein_MS = pd.merge(protein_MS, proteins, how='left', left_on='protein', right_on='ID')
protein_MS['mRNA(M)'].fillna(0, inplace=True)
protein_MS.drop(['ID'], axis=1, inplace=True)

In [83]:
protein_MS

,entry no.,protein,protein probability,protein description,protein length,percent coverage,tot indep spectra,percent share of spectrum ids,peptides,ProteinProphet_Score,mRNA(M)
0,1,YAL003W,1.0,"EFB1 SGDID:S000000003,'Translation elongation ...",0,42.7,312,0.425,AFQSAYPEFSR+AIEMEGLTWGAHQFIPIGFGIK+IETLKQLNASL...,1.00,25.55
1,2,YAL005C,1.0,"SSA1 SGDID:S000000004,'ATPase involved in prot...",0,54.5,838,0.797,AEETISWLDSNTTASKEEFDDKLK+ARFEELCADLFR+ATAGDTHL...,1.00,9.78
2,3,YAL012W,1.0,"CYS3 SGDID:S000000010,'Cystathionine gamma-lya...",0,36.5,235,0.321,AIHAGEHVDVHGSVIEPISLSTTFK+DALGGGMISFR+EASGVFDD...,1.00,8.94
3,4,YAL016W,1.0,"TPD3 SGDID:S000000014,'Regulatory subunit A of...",0,13.9,17,0.021,DSDSNESLYPLALLMDELK+FLNDPSIILNK+ILPAVQNLSMDESE...,0.00,3.12
4,5,YAL021C,1.0,"CCR4 SGDID:S000000019,'Component of the CCR4-N...",0,5.7,8,0.007,DAMDFSGAWMKHKK+FIGFPNDKFPSDHIPLLAR+LSQYSIDEDDD...,0.28,1.39
...,...,...,...,...,...,...,...,...,...,...,...
3848,3821,YPR188C,0.0,"MLC2 SGDID:S000006392,'Regulatory light chain ...",0,0.0,0,0.000,MDHSESLTFNQLTQDYINK,0.00,1.67
3849,3822,YPR195C,0.0,"YPR195C SGDID:S000006399,'Hypothetical protein...",0,0.0,0,0.000,GCDGTVTTFRTWHTQPLGPCNTIIIGR,NaN,0.00
3850,3823,YPR196W,0.0,"YPR196W SGDID:S000006400,'nuclear protein (put...",0,0.0,0,0.000,FFDDLSNSDIFRIMTYYCLHR,NaN,0.00
3851,3824a,YGR032W,0.0,"GSC2 SGDID:S000003264,'Catalytic subunit of 1,...",0,0.0,0,0.000,APTFFVSQDDNNFETEFFPR+DLGFGTILNFTTK+EQTFGTLFAR+...,NaN,0.00


In [74]:
protein_MS.to_csv('../data/yeast-multi-graph/raw/protein/yeast-LCQ.csv')

In [169]:
# need to manually remove extra ID columns